In [31]:
import smtplib
from email.mime.text import MIMEText
from newspaper import Article
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import re
from dotenv import load_dotenv
import os

In [32]:
load_dotenv()  # Load variables from .env

sender_email = os.getenv("sender_email")
sender_password = os.getenv("sender_password")

In [33]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu") #For Apple Silicon Macs
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # For windows users

In [34]:
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn")
model = model.to(device)

In [35]:
summarizer = pipeline("summarization", model=model, tokenizer=tokenizer, device=-1)

Device set to use mps:0


In [ ]:
# Function to clean text
def clean_text(text):
    text = re.sub(r'\s+', ' ', text).strip() #Removes the white space
    return text

In [37]:
def get_summary(url):
    try:
        article = Article(url)
        article.download()
        article.parse()
        text = article.text
        if not text:
            return "Error: Could not extract text from the article."
        
        # Clean and limit text length for summarization
        text = clean_text(text)
        max_input_length = 1024  # BART's max input length
        text = text[:max_input_length]
        
        # Summarize the text
        summary = summarizer(text, max_length=150, min_length=50, do_sample=False)[0]['summary_text']
        return summary
    except Exception as e:
        return f"Error processing article: {str(e)}"

In [38]:
# Function to send email
def send_email(summary, recipient_email, sender_email, sender_password):
    try:
        msg = MIMEText(summary)
        msg['Subject'] = 'Article Summary'
        msg['From'] = sender_email
        msg['To'] = recipient_email

        with smtplib.SMTP('smtp.gmail.com', 587) as server:
            server.starttls()
            server.login(sender_email, sender_password)
            server.sendmail(sender_email, recipient_email, msg.as_string())
        return True
    except Exception as e:
        return f"Error sending email: {str(e)}"

In [41]:
# Function to send email
def send_email(summary, recipient_email, sender_email, sender_password):
    try:
        msg = MIMEText(summary)
        msg['Subject'] = 'Article Summary'
        msg['From'] = sender_email
        msg['To'] = recipient_email

        with smtplib.SMTP('smtp.gmail.com', 587) as server:
            server.starttls()
            server.login(sender_email, sender_password)
            server.sendmail(sender_email, recipient_email, msg.as_string())
        return True
    except Exception as e:
        return f"Error sending email: {str(e)}"

In [39]:
url = "https://timesofindia.indiatimes.com/city/srinagar/jaw-broken-spine-injured-spicejet-staff-attacked-by-passenger-at-srinagar-airport/articleshow/123073510.cms"
summary = get_summary(url)

In [ ]:
email = "tusharrohilla70@gmail.com"
email_result = send_email(summary, email, sender_email, sender_password)

"A senior Army official allegedly 'grievously assaulted' four SpiceJet employees at Srinagar Airport on July 26, 2025. The airline says its staff members ‘suffered a spinal fracture and serious jaw injuries after being attacked with punches, repeated kicks and a queue stand’"